$$\require{mhchem}$$

# Modelling complex reactions (part 1b)
_This lab is based on the material in Fogler, Elements of Chemical Reaction Engineering (2006, 4th ed)_ 

_Revised Oct 2024 to adapt to using Python._

## 2. A more complex reaction system

**We will now consider a more complex system involving four competing reactions. The stoichiometries and rate equations are also more complex. You will now construct and solve the ODE system with less assistance, assuming that these reactions take place in a <u>plug-flow reactor (PFR)</u>** 

### 2.1 The reaction mechanism
Assume that the follow reactions take place concurrently in a PFR:
 
Reaction 1: &nbsp; &nbsp; &nbsp; &nbsp; $\ce{4A + 5B ->[k_{1A}] 4C + 6D}$ &nbsp; &nbsp; &nbsp; &nbsp; $r_{1A} = -k_{1A}C_A C_B^2$ 

Reaction 2: &nbsp; &nbsp; &nbsp; &nbsp; $\ce{2A + \frac{3}{2}B ->[k_{2A}] E + 3D}$ &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; $r_{2A} = -k_{2A}C_A C_B$ 

Reaction 3: &nbsp; &nbsp; &nbsp; &nbsp; $\ce{2C + B ->[k_{3B}] 2F}$ &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;&nbsp; &nbsp; &nbsp;  &nbsp; &nbsp; &nbsp; $r_{3B} = -k_{3B}C_C^2 C_B$ 

Reaction 4: &nbsp; &nbsp; &nbsp; &nbsp; $\ce{4A + 6C ->[k_{4C}] 5E + 6D}$ &nbsp; &nbsp; &nbsp; &nbsp; $r_{4C} = -k_{4C}C_A^{1.5} C_C$ 

A number of things to notice about this reaction mechanism: 
* Certain species are involved in multiple reactions, as reactants and/or products. 
* The orders of reaction with respect to each species do not always correspond to the species’ stoichiometric coefficient (e.g. Reaction 1). This means that the reactions shown are not elementary processes, but rather the net reaction of more complex mechanisms. 
*  The rate equation for each reaction is defined according to a specific species. The subscript refers to the reaction number and the species. For example, in Reaction 3, the rate equation is expressed as the consumption of B, hence $k_{3B}$ and $r_{3B}$. 

**What we want to do is to model the reactions taking place in the PFR.**

### 2.1 The kinetic model: PFR in the liquid phase
We consider first the case of the above reactions in the liquid phase. Since they take place in a PFR, we can use the molar material balance equation for a PFR to give us the differential equations for how molar flow rates change as a function of reactor volume. For a species $i$:

$$\frac{d\dot n_i}{dV} = r_i$$

where $r_i$ is the rate equation for species $i$ taking into account all the reactions which it is involved in.

Writing such an equation for every species (i.e. $\frac{d\dot n_A}{dV} = r_i$, $\frac{d\dot n_B}{dV} = r_i$...) gives us then the system of ODEs that we need to solve.

**Question: What is the independent variable when we model a PFR? What is the data/information that we get after we solve such a system of ODEs?**

Answer:




To complete the equations, we need substitute $r_A$, $r_B$… with the actual rate equations. For example, for the species E:

$$\frac{d\dot n_E}{dV} = r_E = r_{2E} + r_{4E}$$

$$\Rightarrow \frac{d\dot n_E}{dV} = (-\frac{1}{2} r_{2A}) + (-\frac{5}{6} r_{4C})$$

Note the adjustments to the coefficients due to stoichiometry and reactant/product relationships

$$\Rightarrow \frac{d\dot n_E}{dV} = \frac{1}{2} k_{2A} C_A C_B + \frac{5}{6} k_{4C} C_C C_A^{1.5}$$

It can be seen that while the molar flow rate is the dependent variable on the left-hand side, the right-hand side is expressed in terms of concentration. The easiest way to resolve this for a liquid-phase system (where the volumetric flow rate $\upsilon_0$ is conststant) is to realise that: 

$$\dot n_i = C_i \upsilon_0 \Rightarrow \frac{d\dot n_E}{dV} = \upsilon_0 \frac{dC_i}{dV}$$

Then:
$$\Rightarrow \frac{dC_E}{dV} = (-\frac{1}{2} r_{2A} -\frac{5}{6} r_{4C})/\upsilon_0$$

or
$$\Rightarrow \frac{dC_E}{dV} = (\frac{1}{2} k_{2A} C_A C_B + \frac{5}{6} k_{4C} C_C C_A^{1.5})/\upsilon_0$$

**Task: Construct the system of ODEs to describe the reactions in the PFR shown above. Take extra care with the signs of the terms and the coefficients relating to reaction stoichiometry!**

Answer:










    

### 2.2 Solving the ODE model with Python


Now we need to write a script in Python to solve this kinetics model, using one of the ODE solvers. Use the example of the consecutive reactions mechanism to help you do this. Include a plot how the concentrations of the species changee along the PFR.

Some lines of code and comments are given below to help get you started. We'll call this function `compete`, since there are competing reactions in this reaction mechanism.

Use the following reaction parameters and initial conditions: 

$k_{1A} = 5.0 (m^3/kmol)^2/min;k_{2A} = 2.0 m^3/(kmol\cdot min)$

$k_{3B} = 10.0 (m^3/kmol)^2/min;k_{4C} = 5.0 (m^3/kmol)^{2/3}/min$

$C_{A0} = C_{B0} = 1.0 mol/dm3$

$C_{C0} = C_{D0} = C_{E0} = C_{F0} = 0$

$\upsilon_0 = 10 dm^3/min$

PFR volume $V = 10 dm^3$


In [ ]:
# Using solve_ivp
# import what you will need from relevant libraries.
import numpy as np   
from scipy.integrate import solve_ivp 
import matplotlib.pyplot as plt

# Define the system of ODEs as your kinetics model to be solved.
# solve_ivp to be used -> indep variable comes first
def compete(V, concentrations):
    A, B, C, D, E, F = concentrations

    # Define the rate laws for the individual reactions - fill in the rest!
    r1A = -k1A * A * B**2
    r2A = -k2A * A * B   # rate law for Reaction 2
    r3B = -k3B * C**2 * B
    r4C = -k4C * C * A**1.5   #rate law for Reaction 4
        
    # Define the rate laws for the different species in the reaction - fill in the rest!
    dA_dV = (r1A + r2A + (4/6)*r4C) / u0
    dB_dV = ((5/4)*r1A + (1.5/2)*r2A + r3B) / u0
    dC_dV = (-r1A + 2*r3B + r4C) / u0
    dD_dV = (-(6/4)*r1A - (3/2)*r2A - r4C) / u0
    dE_dV = (-(1/2)*r2A -(5/6)*r4C) / u0
    dF_dV = (-2*r3B) / u0
    return [dA_dV, dB_dV, dC_dV, dD_dV, dE_dV, dF_dV]

# Define the constants
k1A = 5.0
k2A = 2.0
k3B = 10.0
k4C = 5.0
u0 = 10.0

# Define the initial concentrations
A0 = 1.0
B0 = 1.0
C0 = 0.0
D0 = 0.0
E0 = 0.0
F0 = 0.0

# Define the volume span for the ODE solution
V_span = (0, 10)

# Solve using solve_ivp
solution = solve_ivp(compete, V_span, [A0, B0, C0, D0, E0, F0], dense_output = True)
print(solution) # not necessary, but a useful check when starting out

V = solution.t   # this extracts the x-values (named t by solve_ivp, as it's assumed to be time points)
concentrations = solution.y # this extracts the x-values (named y by solve_ivp)

# Plot the results
plt.plot(V, concentrations[0], label='[A]')  
plt.plot(V, concentrations[1], label='[B]')
plt.plot(V, concentrations[2], label='[C]')
plt.plot(V, concentrations[3], label='[D]')
plt.plot(V, concentrations[4], label='[E]')
plt.plot(V, concentrations[5], label='[F]')
plt.xlabel('Volume')
plt.ylabel('Concentration')
plt.legend()
plt.title('Concentration vs volume for a complex competing reaction mechanism')
plt.show()


In [ ]:
# Using odeint
# import what you will need from relevant libraries.
import numpy as np   
from scipy.integrate import odeint 
import matplotlib.pyplot as plt

# Define the system of ODEs as your kinetics model to be solved.
# odeint to be used -> dep variable comes first
def compete(concentrations, V):
    A, B, C, D, E, F = concentrations

    # Define the rate laws for the individual reactions - fill in the rest!
    r1A = -k1A * A * B**2
    r2A = -k2A * A * B   # rate law for Reaction 2
    r3B = -k3B * C**2 * B
    r4C = -k4C * C * A**1.5   #rate law for Reaction 4
        
    # Define the rate laws for the different species in the reaction - fill in the rest!
    dA_dV = (r1A + r2A + (4/6)*r4C) / u0
    dB_dV = ((5/4)*r1A + (1.5/2)*r2A + r3B) / u0
    dC_dV = (-r1A + 2*r3B + r4C) / u0
    dD_dV = (-(6/4)*r1A - (3/2)*r2A - r4C) / u0
    dE_dV = (-(1/2)*r2A -(5/6)*r4C) / u0
    dF_dV = (-2*r3B) / u0
    return [dA_dV, dB_dV, dC_dV, dD_dV, dE_dV, dF_dV]

# Define the constants
k1A = 5.0
k2A = 2.0
k3B = 10.0
k4C = 5.0
u0 = 10.0

# Define the initial concentrations
A0 = 1.0
B0 = 1.0
C0 = 0.0
D0 = 0.0
E0 = 0.0
F0 = 0.0

# Define the volume points for the ODE solution
V = np.linspace(0, 10, 100)  
print(V)

# Solve using odeint
solution = odeint(compete, [A0, B0, C0, D0, E0, F0], V)
print(solution) # not necessary, but a useful check when starting out

# Plot the results
plt.plot(V, solution[:, 0], label='[A]')  
plt.plot(V, solution[:, 1], label='[B]')
plt.plot(V, solution[:, 2], label='[C]')
plt.plot(V, solution[:, 3], label='[D]')
plt.plot(V, solution[:, 4], label='[E]')
plt.plot(V, solution[:, 5], label='[F]')
plt.xlabel('Volume')
plt.ylabel('Concentration')
plt.legend()
plt.title('Concentration vs volume for a complex competing reaction mechanism')
plt.show()


**Question: Look at how [C] changes with reactor volume. Why does it change the way it does?**

Answer:





**Question: What volume should the PFR have to maximise the yield of C? Determine this using Python!**

In [ ]:
#Answer:






**Question: If E were the desired product, and C and F were unwanted side products, would it be better to double the concentration of reactant A or reactant B in the feed to the reactor?**

**Predict this first with reference to the kinetics modelwithout changing your code, then test this with a new calculation. Discuss and explain how your prediction compared to the calculated results.**

Answer:





